<a href="https://colab.research.google.com/github/Crisitunity-Lab/ARDC-Project/blob/main/Notebooks/Llama2_prod_r3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Llama2 Production

SUMA 26/09/2023

#Instructions

Running code on Google Colab
1.   Create HuggingFace account
2.   Gain access to model: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
3.   Create folder 'data' in project folder on personal Google Drive
4.   Save data set CrisiLexT26 in folder 'data'
5.   Data sourced from: https://crisislex.org/data-collections.html#CrisisLexT26
6.   Mount Google Drive
7.   Update code with Paths
8.   Set system up as per steps below
9.   Run Experiments
10.  Evaluate

## Setup

In [1]:
# Install Transformer package
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00


In [2]:
# Import Packages
from langchain import PromptTemplate,  LLMChain
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import pandas as pd
import numpy as np
import json
import os
import csv
import time
import re
import matplotlib.pyplot as plt
import json



In [3]:
# Mounting GDrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Prerequisites

To load our desired model, `meta-llama/Llama-2-7b-chat-hf`, we first need to authenticate ourselves on Hugging Face. This ensures we have the correct permissions to fetch the model.

1. Gain access to the model on Hugging Face: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).
2. Use the Hugging Face CLI to login and verify your authentication status.


In [4]:
# Logging onto HuggingFace
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00


## Preparing data

In [6]:
# Read (timestamps)

# Path to the main folder
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Function to process CSV files in a folder
def process_folder(folder_path):
    dfs = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv') and 'period' in file.lower():
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                dfs.append(df)
    return dfs

# Read and process each subfolder
combined_data = []
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        subfolder_data = process_folder(subfolder_path)
        combined_data.extend(subfolder_data)

# Concatenate all data into one DataFrame
combined_df_p = pd.concat(combined_data, ignore_index=True)


# Remove spaces from column names
combined_df_p.columns = combined_df_p.columns.str.replace(' ', '')

# Rename Columns in prep for left join
combined_df_p.rename(columns={'Tweet-ID': 'Tweet ID'}, inplace=True)

# # Save the combined DataFrame to a CSV file
# output_file = 'combined_data.csv'
# combined_df.to_csv(output_file, index=False)

print(f"Combined timestamps read")


Combined timestamps read


In [7]:
# Read tweets

# Path to the main folder
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Function to process CSV files in a folder, add folder name as label, and add subfolder name as a column
def process_folder(folder_path, label, subfolder_name):
    dfs = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv') and 'labeled' in file.lower():
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                df['Label'] = label
                df['subfolder_name'] = subfolder_name  # Add subfolder name as a column
                dfs.append(df)
    return dfs

# Read and process each subfolder
combined_data = []
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        label = subfolder
        subfolder_data = process_folder(subfolder_path, label, subfolder)
        combined_data.extend(subfolder_data)

# Concatenate all data into one DataFrame
combined_df_l = pd.concat(combined_data, ignore_index=True)

# Print the first few rows of the combined DataFrame
print(f"Combined tweets read")
combined_df_l.head(10)


Combined tweets read


,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,Label,subfolder_name
0,347686624563429378,"RT @CBCAlerts: Canmore, Alta. declares state o...",Media,Affected individuals,Related and informative,2013_Alberta_floods,2013_Alberta_floods
1,347766337344503808,RT @GlobalCalgary: If you are in #Canmore and ...,Media,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods
2,347779159327637504,RT @metrocalgary: UPDATE: Latest from the @cit...,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods
3,347783236191129600,RT @GlobalCalgary: GALLERY: Incredible photos ...,Media,Not applicable,Related and informative,2013_Alberta_floods,2013_Alberta_floods
4,347793432514801665,RT @nenshi: Major risk of flooding in Calgary....,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods
5,347793818403344384,RT @weathernetwork: RT: Residents of #HighRive...,Media,Affected individuals,Related and informative,2013_Alberta_floods,2013_Alberta_floods
6,347802211222433792,RT @BrockWHarrison: . @ElectDanielle and other...,NGOs,Affected individuals,Related and informative,2013_Alberta_floods,2013_Alberta_floods
7,347804916514951168,Lots of #abflood updates on our liveblog http:...,Media,Other Useful Information,Related and informative,2013_Alberta_floods,2013_Alberta_floods
8,347808955663257600,RT @aldjohnmar: Our first Emergency Shelter wi...,NGOs,Infrastructure and utilities,Related and informative,2013_Alberta_floods,2013_Alberta_floods
9,347812738900320256,'Move to high ground:' Rapid flooding forces e...,Media,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods


In [ ]:
# Check
combined_df_p.columns
combined_df_p.head()


,Timestamp,Tweet ID,Included(Y/N)
0,Thu Jun 20 03:56:19 +0000 2013,347563538547609602,Y
1,Mon Jul 01 16:10:43 +0000 2013,351734622884855808,Y
2,Mon Jul 01 16:19:46 +0000 2013,351736900391931904,Y
3,Mon Jul 01 16:28:15 +0000 2013,351739035271700482,Y
4,Mon Jul 01 16:38:38 +0000 2013,351741648327294977,Y


In [8]:
# Extract earliest timestamp

# Convert it to datetime first
combined_df_p['Timestamp'] = pd.to_datetime(combined_df_p['Timestamp'])

# Group by 'Tweet-ID' and aggregate to find the earliest timestamp
earliest_timestamps = combined_df_p.groupby('Tweet ID')['Timestamp'].agg('min').reset_index()

# Display or use the resulting DataFrame 'earliest_timestamps'
# print(earliest_timestamps)

In [9]:
# Left join

# Perform a left join on 'Tweet ID'
result_df = combined_df_l.merge(earliest_timestamps, on='Tweet ID', how='left')

# Display
# print(result_df)
# result_df.shape
result_df.head()

,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,Label,subfolder_name,Timestamp
0,347686624563429378,"RT @CBCAlerts: Canmore, Alta. declares state o...",Media,Affected individuals,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 12:05:25+00:00
1,347766337344503808,RT @GlobalCalgary: If you are in #Canmore and ...,Media,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 17:22:10+00:00
2,347779159327637504,RT @metrocalgary: UPDATE: Latest from the @cit...,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 18:13:07+00:00
3,347783236191129600,RT @GlobalCalgary: GALLERY: Incredible photos ...,Media,Not applicable,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 18:29:19+00:00
4,347793432514801665,RT @nenshi: Major risk of flooding in Calgary....,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 19:09:50+00:00


In [10]:
# Read

# Define the main folder path
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Initialize empty lists to store the extracted data
subfolder_names = []
names = []
start_days = []
durations = []
countries = []
location_descriptions = []
sub_categories = []
types = []

# Loop through the subfolders
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)

    # Check if it's a directory
    if os.path.isdir(subfolder_path):
        json_files = [f for f in os.listdir(subfolder_path) if f.endswith('.json')]

        # Loop through JSON files in the subfolder
        for json_file in json_files:
            json_path = os.path.join(subfolder_path, json_file)

            # Read the JSON file
            with open(json_path, 'r') as f:
                data = json.load(f)

                # Extract the required information
                subfolder_names.append(subfolder)
                names.append(data['name'])
                start_days.append(data['time']['start_day'])
                durations.append(data['time']['duration'])
                countries.append(data['location']['country'])
                location_descriptions.append(data['location']['location_description'])
                sub_categories.append(data['categorization']['sub_category'])
                types.append(data['categorization']['type'])

# Create a DataFrame from the extracted data
df_annotation = pd.DataFrame({
    'subfolder_name': subfolder_names,
    'name': names,
    'start_day': start_days,
    'duration': durations,
    'country': countries,
    'location_description': location_descriptions,
    'sub_category': sub_categories,
    'type': types
})

# Display the DataFrame
df_annotation.head()


,subfolder_name,name,start_day,duration,country,location_description,sub_category,type
0,2013_Alberta_floods,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
1,2013_Glasgow_helicopter_crash,Glasgow helicopter crash,2013-11-29,30,UK,Glasgow,Unintentional,Crash
2,2012_Costa_Rica_earthquake,Costa Rica earthquake,2012-09-04,13,Costa Rica,Costa Rica,Geophysical,Earthquake
3,2013_Manila_floods,Manila Floods,2013-08-17,11,Phillipines,Manila,Hydrological,Floods
4,2013_LA_airport_shootings,LA Airport Shootings,2013-11-01,12,US,Los Angeles,Intentional,Shootings


In [11]:
# Left join

# Perform a left join on the 'subfolder'
result_df = result_df.merge(df_annotation, left_on='subfolder_name', right_on='subfolder_name', how='left')

# Drop the duplicate 'subfolder_name'
result_df.drop(columns=['subfolder_name'], inplace=True)


In [12]:
# Count the number of rows in result_df
row_count = result_df.shape[0]

# Print the row count
print(f"Number of rows in result_df: {row_count}")

result_df.head()


Number of rows in result_df: 27933


,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,Label,Timestamp,name,start_day,duration,country,location_description,sub_category,type
0,347686624563429378,"RT @CBCAlerts: Canmore, Alta. declares state o...",Media,Affected individuals,Related and informative,2013_Alberta_floods,2013-06-20 12:05:25+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
1,347766337344503808,RT @GlobalCalgary: If you are in #Canmore and ...,Media,Caution and advice,Related and informative,2013_Alberta_floods,2013-06-20 17:22:10+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
2,347779159327637504,RT @metrocalgary: UPDATE: Latest from the @cit...,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013-06-20 18:13:07+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
3,347783236191129600,RT @GlobalCalgary: GALLERY: Incredible photos ...,Media,Not applicable,Related and informative,2013_Alberta_floods,2013-06-20 18:29:19+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
4,347793432514801665,RT @nenshi: Major risk of flooding in Calgary....,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013-06-20 19:09:50+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods


## Modeling

In [13]:
def generate_answer(text, system_prompt, instruction):
  promt_template = get_prompt(instruction, system_prompt)
  prompt = PromptTemplate(template=prompt_template, input_variables= ['text'])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  text = llm_chain.run(text)
  return text

In [14]:
# Tokenizer
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
# Load model
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


In [16]:
# Prompt Template Formatting
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text

In [17]:
data = {
    'TweetID': [25454],
    'TweetText': ['MT @COEmergency Colorado 2-1-1 Recovery and Relief Guides http://t.co/VyeHrWp5Un … #COflood #hmrd ^CB'],
    'InformationSource': ['Government'],
    'InformationType': ['Donations and volunteering'],
    'Informativeness Label': ['Related and informative'],
    'Timestamp': ['2013-09-17 10:43:12+00:00'],
    'answer': ['']
}

df = pd.DataFrame(data)
df


,TweetID,TweetText,InformationSource,InformationType,Informativeness Label,Timestamp,answer
0,25454,MT @COEmergency Colorado 2-1-1 Recovery and Re...,Government,Donations and volunteering,Related and informative,2013-09-17 10:43:12+00:00,


In [18]:
# Create a function using the code below to generate a answer for each row in the dataframe. Then apply the function to the dataframe to generate the answer and save the output to 'answer' in df.
def generate_answer(text, system_prompt, instruction):
    # # Define the instruction
    # instruction = instruction + row['TweetText']
    #define the prompt template
    prompt_template = get_prompt(instruction, system_prompt)
    #create prompt
    prompt = PromptTemplate(template=prompt_template, input_variables= ["text"])
    #create the language model chain
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    #generate text using the prompt template
    text = llm_chain.run(text)
    #return the text
    return text


In [19]:
# Apply the function to the dataframe to generate the answer and save the output to 'answer' in df.
system_prompt = "You are an advance assistance that excels at classifying topics. Stop after answering question."
instruction = "Classify the following text for me {text}"
df['answer'] = df.apply(lambda row: generate_answer(row['TweetText'], system_prompt, instruction), axis=1)

In [20]:
# Check
df.head()


,TweetID,TweetText,InformationSource,InformationType,Informativeness Label,Timestamp,answer
0,25454,MT @COEmergency Colorado 2-1-1 Recovery and Re...,Government,Donations and volunteering,Related and informative,2013-09-17 10:43:12+00:00,"Sure! Based on the provided text, I would cl..."


In [21]:
# Grab 10 random samples
filtered_df = result_df.query("Label == '2013_Colorado_floods'")
testing_df = filtered_df.sample(n=10, random_state=42)
testing_df.columns = testing_df.columns.str.replace(' ', '')
testing_df['TweetText'] = testing_df['TweetText'].astype(str)


testing_df.head(12)


,TweetID,TweetText,InformationSource,InformationType,Informativeness,Label,Timestamp,name,start_day,duration,country,location_description,sub_category,type
10507,379918454104408064,MT @COEmergency Colorado 2-1-1 Recovery and Re...,Government,Donations and volunteering,Related and informative,2013_Colorado_floods,2013-09-17 10:43:12+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods
10723,380796468728045568,"$CLF Factbox: Floods shut oil, gas operations ...",Media,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-19 20:52:07+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods
10726,380808204420014080,Wow- plowing out the mud from the bike path I ...,Eyewitness,Infrastructure and utilities,Related - but not informative,2013_Colorado_floods,2013-09-19 21:38:45+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods
10646,380450807746797568,Proud to be playing this all star event in #Lo...,Outsiders,Donations and volunteering,Related - but not informative,2013_Colorado_floods,2013-09-18 21:58:35+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods
10397,379623338672287744,RT @OWHnews: Colorado flooding: Have your frie...,Media,Affected individuals,Related - but not informative,2013_Colorado_floods,2013-09-16 15:10:31+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods
10664,380490196464041984,#Jamestown resident says things are disappeari...,Outsiders,Affected individuals,Related and informative,2013_Colorado_floods,2013-09-19 00:35:06+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods
10612,380337729331933184,RT @EnviroAction: Colorado #fracking site feat...,NGOs,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-18 14:29:15+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods
10499,379836589691506688,Praying for Colorado because its crazy floodin...,Outsiders,Sympathy and support,Related - but not informative,2013_Colorado_floods,2013-09-17 05:17:54+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods
10845,381803449840435200,RT @denverpost: Colorado's September floods wi...,Media,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-22 15:33:30+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods
10122,378191675941588992,"RT @sciam: Two Dead, Hundreds Evacuated, Cars ...",Media,Affected individuals,Related and informative,2013_Colorado_floods,2013-09-12 16:21:36+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods


In [24]:
# Running over data

system_prompt = "You are an advance assistance that excels at classifying topics. Stop after answering question."
instruction = "Classify the following text for me {text}"

# Create an empty 'answer' column in testing_df
testing_df['answer'] = ''

# Loop over the data
for index, row in testing_df.iterrows():
    TweetText = row['TweetText']

    # Generate the prompt
    # prompt_template = get_prompt(instruction, system_prompt)

    # Update the 'answer' column in the DataFrame
    testing_df.at[index, 'answer'] = df.apply(lambda row: generate_answer(row['TweetText'], system_prompt, instruction), axis=1)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentiall

In [25]:
# Check
testing_df.head()


,TweetID,TweetText,InformationSource,InformationType,Informativeness,Label,Timestamp,name,start_day,duration,country,location_description,sub_category,type,answer
10507,379918454104408064,MT @COEmergency Colorado 2-1-1 Recovery and Re...,Government,Donations and volunteering,Related and informative,2013_Colorado_floods,2013-09-17 10:43:12+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods,"0 Sure! Based on the provided text, I wou..."
10723,380796468728045568,"$CLF Factbox: Floods shut oil, gas operations ...",Media,Infrastructure and utilities,Related and informative,2013_Colorado_floods,2013-09-19 20:52:07+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods,"0 Sure! Based on the provided text, I wou..."
10726,380808204420014080,Wow- plowing out the mud from the bike path I ...,Eyewitness,Infrastructure and utilities,Related - but not informative,2013_Colorado_floods,2013-09-19 21:38:45+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods,"0 Sure! Based on the provided text, I wou..."
10646,380450807746797568,Proud to be playing this all star event in #Lo...,Outsiders,Donations and volunteering,Related - but not informative,2013_Colorado_floods,2013-09-18 21:58:35+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods,"0 Sure! Based on the provided text, I wou..."
10397,379623338672287744,RT @OWHnews: Colorado flooding: Have your frie...,Media,Affected individuals,Related - but not informative,2013_Colorado_floods,2013-09-16 15:10:31+00:00,Colorado Floods,2013-09-08,21,US,Colorado,Hydrological,Floods,"0 Sure! Based on the provided text, I wou..."


## Experiments

## Performance Evaluation